## Problema de Transporte

In [1]:
import pyomo.environ as pyEnv
import numpy as np

In [2]:
model = pyEnv.ConcreteModel()

In [3]:
model.suppliers = pyEnv.RangeSet(2)
model.consumers = pyEnv.RangeSet(3)

In [4]:
# Definindo custo
cost = np.array([3, 5, 7, 3, 2, 5]).reshape(2,3)
model.cost = pyEnv.Param(model.suppliers, model.consumers, initialize = lambda model, i, j: cost[i-1][j-1])

capacity = np.array([25, 35])
model.capacity = pyEnv.Param(model.suppliers, initialize = lambda model, i: capacity[i-1])

demand = [20, 30, 10]
model.demand = pyEnv.Param(model.consumers, initialize = lambda model, j: demand[j-1])

In [5]:
# Definindo as variáveis x
model.x = pyEnv.Var(model.suppliers, model.consumers, within = pyEnv.NonNegativeReals)

In [6]:
# Função Objetivo
def obj(model):
    return sum(model.cost[i,j] * model.x[i,j] for i in model.suppliers for j in model.consumers)

model.obj = pyEnv.Objective(rule = obj, sense = pyEnv.minimize)

In [7]:
# Restrição de Demanda
def capacity_cons(model, i):
    return sum(model.x[i,j] for j in model.consumers) <= model.capacity[i]

model.capacity_cons = pyEnv.Constraint(model.suppliers, rule = capacity_cons)

In [8]:
# Restrição de Capacidade
def demand_cons(model, j):
    return sum(model.x[i,j] for i in model.suppliers) == model.demand[j]

model.demand_cons = pyEnv.Constraint(model.consumers, rule = demand_cons)

In [9]:
# Resolvendo o problema
path = '/home/helano-pessoa/Softwares/Cplex-12.10/cplex/bin/x86-64_linux/cplex'
opt = pyEnv.SolverFactory("cplex", solver_io = "python", executable = path)
results = opt.solve(model)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: 
  Lower bound: 180.0
  Upper bound: 180.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 6
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 6
  Number of nonzeros: None
  Sense: 1
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Name: CPLEX 12.10.0.0
  Status: ok
  Wallclock time: 0.0005705356597900391
  Termination condition: optimal
# ----------------------------------------------------------
#   Solution Information
# -----------------------------------------

In [10]:
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 3*x[1,1] + 5*x[1,2] + 7*x[1,3] + 3*x[2,1] + 2*x[2,2] + 5*x[2,3]


In [11]:
model.capacity_cons.pprint()

capacity_cons : Size=2, Index=suppliers, Active=True
    Key : Lower : Body                     : Upper : Active
      1 :  -Inf : x[1,1] + x[1,2] + x[1,3] :  25.0 :   True
      2 :  -Inf : x[2,1] + x[2,2] + x[2,3] :  35.0 :   True


In [12]:
model.demand_cons.pprint()

demand_cons : Size=3, Index=consumers, Active=True
    Key : Lower : Body            : Upper : Active
      1 :  20.0 : x[1,1] + x[2,1] :  20.0 :   True
      2 :  30.0 : x[1,2] + x[2,2] :  30.0 :   True
      3 :  10.0 : x[1,3] + x[2,3] :  10.0 :   True


In [13]:
model.x.display()

x : Size=6, Index=x_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1) :     0 :  20.0 :  None : False : False : NonNegativeReals
    (1, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (1, 3) :     0 :   5.0 :  None : False : False : NonNegativeReals
    (2, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (2, 2) :     0 :  30.0 :  None : False : False : NonNegativeReals
    (2, 3) :     0 :   5.0 :  None : False : False : NonNegativeReals


In [14]:
model.obj.display()

obj : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 180.0
